In [ ]:
!pip install noise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for noise: filename=noise-1.2.2-cp310-cp310-linux_x86_64.whl size=56278 sha256=8ededd3e33a3b17b124847ba4689bd4a4b9dfc6389b717e0af6221a6205aa82b
  Stored in directory: /root/.cache/pip/wheels/b2/01/64/ca2f6ed556c204d5ce2d3719a58757a5504aeb5525654692d0
Successfully built noise


In [1]:
import numpy as np
import pandas as pd
from noise import pnoise2
import altair as alt

# Generate a ring pattern
size = 50
x = np.linspace(-1, 1, size)
y = np.linspace(-1, 1, size)
X, Y = np.meshgrid(x, y)

a = 0
b = -0.2

# Create a ring pattern using distance from center
R = np.sqrt((X-a)**2 + (Y-b)**2)
ring_width = 0.5
ring = np.zeros((size, size))

# Create a ring with smooth edges
inner_radius = 0.5
outer_radius = 0.7
ring = 1 - np.abs((R - (inner_radius + outer_radius)/2) / (ring_width/2))
ring = np.clip(ring, 0, 1)  # Clip values between 0 and 1



# Generate Perlin noise
noise_scale = 5.0  # Lower values = higher frequency
octaves = 4  # More octaves = more detail
persistence = 0.5  # How much each octave contributes
lacunarity = 2.0  # How much detail is added in each octave
base = 0  # Random seed

noise = np.zeros(ring.shape)
for i in range(size):
    for j in range(size):
        noise[i][j] = pnoise2(i/noise_scale,
                             j/noise_scale,
                             octaves=octaves,
                             persistence=persistence,
                             lacunarity=lacunarity,
                             base=base)

# Normalize noise to [0,1] range
noise = (noise - noise.min()) / (noise.max() - noise.min())

# Combine ring and noise
noise_strength = 0.3  # Adjust this to control how much noise affects the image
combined = ring + noise_strength * noise
combined = (combined - combined.min()) / (combined.max() - combined.min())

ModuleNotFoundError: No module named 'noise'

In [ ]:
zero_grid = np.zeros((size, size))

photodetector_locations = np.array([
    (0, 15),
    (1.756, 8.827),
    (3.512, 17.654),
    (5.74, 13.858),
    (7.483, 5),
    (10.607, 10.607),
    (14.966, 10),
    (13.858, 5.74),
    (15, 0),
    (8.827, -1.756),
    (17.654, -3.512),
    (13.858, -5.740),
    (5, -7.483),
    (10.607, -10.607),
    (10, -14.966),
    (5.74, -13.858),
    (0, -15),
    (-1.756, -8.827),
    (-3.512, -17.654),
    (-5.74, -13.858),
    (-7.483, -5),
    (-10.607, -10.607),
    (-14.966, -10),
    (-13.858, -5.74),
    (-15, 0),
    (-8.827, 1.756),
    (-17.654, 3.512),
    (-13.858, 5.740),
    (-5, 7.483),
    (-10.607, 10.607),
    (-10, 14.966),
    (-5.74, 13.858)
])

detector_size = 40

photodetector_locations = np.round((detector_size*photodetector_locations / (2*photodetector_locations.max())) + (combined.shape[0]/2, combined.shape[0]/2))

# Create a DataFrame for plotting
df = pd.DataFrame(photodetector_locations, columns=['x', 'y'])

# Create scatter plot using Altair
chart = alt.Chart(df).mark_circle(
    size=50,  # Size of the points
    color='blue',
    opacity=0.6
).encode(
    x=alt.X('x:Q',
            scale=alt.Scale(domain=[0, combined.shape[0]]),  # Adjust domain to fit all points
            axis=alt.Axis(grid=True)),
    y=alt.Y('y:Q',
            scale=alt.Scale(domain=[0, combined.shape[0]]),  # Adjust domain to fit all points
            axis=alt.Axis(grid=True)),
    tooltip=['x', 'y']
).properties(
    width=500,
    height=500,
    title='Photodetector Locations'
).configure_view(
    strokeWidth=1
)

# Save the chart to an HTML file
chart.save('photodetector_locations.html')

display(chart)

alt.Chart(...)

In [ ]:
# Convert data to long format for Altair
df = pd.DataFrame(
    [(i, j, combined[i, j]) for i in range(size) for j in range(size)],
    columns=['x', 'y', 'value']
)

# Create heatmap using Altair with removed axis labels
chart = alt.Chart(df).mark_rect().encode(
    x=alt.X('x:O',
            title='X Axis',
            axis=alt.Axis(labels=False, ticks=False)),  # Remove x-axis labels and ticks
    y=alt.Y('y:O',
            title='Y Axis',
            sort='-y',
            axis=alt.Axis(labels=False, ticks=False)),  # Remove y-axis labels and ticks
    color=alt.Color('value:Q',
                   scale=alt.Scale(scheme='magma')),
    tooltip=['x', 'y', 'value']
).properties(
    width=400,
    height=400,
    title='2D Heatmap Visualization'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

# Save the chart to an HTML file
chart.save('heatmap.html')


display(chart)

alt.Chart(...)

In [ ]:
df_points = pd.DataFrame(photodetector_locations, columns=['x', 'y'])

df_heatmap = pd.DataFrame(
    [(i, j, combined[i, j]) for i in range(size) for j in range(size)],
    columns=['x', 'y', 'value']
)

# Create base heatmap
heatmap = alt.Chart(df_heatmap).mark_rect().encode(
    x=alt.X('x:O',
            title=None,
            axis=alt.Axis(labels=False, ticks=False)),
    y=alt.Y('y:O',
            title=None,
            sort='-y',
            axis=alt.Axis(labels=False, ticks=False)),
    color=alt.Color('value:Q',
                   scale=alt.Scale(scheme='magma')),
    tooltip=['x', 'y', 'value']
)

# Create points layer
points = alt.Chart(df_points).mark_circle(
    size=50,  # Adjust point size as needed
    color='#17becf',  # Point color
    opacity=0.8  # Point opacity
).encode(
    x=alt.X('x:Q',
            scale=alt.Scale(domain=[0, size])),
    y=alt.Y('y:Q',
            scale=alt.Scale(domain=[0, size])),
    tooltip=['x', 'y']
)

# Combine the layers
combined_chart = (heatmap + points).properties(
    width=400,
    height=400,
    title='Heatmap with Photodetector Locations'
).configure_view(
    strokeWidth=0
)

# Save the combined chart
combined_chart.save('combined_heatmap_points.html')

display(combined_chart)

alt.LayerChart(...)

In [ ]:
for point in photodetector_locations:
    print(point, combined[int(point[0])][int(point[1])])

[25. 42.] 0.4915012883940874
[27. 35.] 0.7141289423340944
[29. 45.] 0.11033525613284251
[32. 41.] 0.11601504514196365
[33. 31.] 0.810343188272745
[37. 37.] 0.15787433893180766
[42. 36.] 0.06897599254185866
[41. 32.] 0.09506692760320136
[42. 25.] 0.10346309191686166
[35. 23.] 0.913277626273265
[45. 21.] 0.13081966442434592
[41. 18.] 0.1299760877980585
[31. 17.] 0.781750276926092
[37. 13.] 0.0915601130669869
[36.  8.] 0.07868841886924202
[32.  9.] 0.250693913188081
[25.  8.] 0.5797818108979328
[23. 15.] 0.34178333269591626
[21.  5.] 0.2847389012087619
[18.  9.] 0.8204633476415092
[17. 19.] 0.09743103403763165
[13. 13.] 0.7628364470418938
[ 8. 14.] 0.7711444262890577
[ 9. 18.] 0.6458757827223858
[ 8. 25.] 0.5371286109544767
[15. 27.] 0.08314986233038528
[ 5. 29.] 0.8942318285461887
[ 9. 32.] 0.644713788889486
[19. 33.] 0.09509654438074429
[13. 37.] 0.9062228023524822
[14. 42.] 0.41120216770635504
[18. 41.] 0.7504505685114075
